In [1]:
from common import *
from analysis.potential import load_potential

if not os.path.exists(data_path):
    os.makedirs(data_path)

In [2]:
from dataclasses import dataclass
from typing import Callable, Optional

@dataclass
class SpinOne:
    population_1: float
    phase_proj_1: float
    population_minus_1: float
    phase_proj_minus_1: float

def prepare(j_init: int,
    omega_init: int | SpinOne,
    j_tot: int,
    time_step: float = 200,
    steps_no: float = 300,
    r_start: float = 50 / 1024,
    r_end: float = 50,
    r_no: int = 1024,
    polar_no: int = 160, 
    omega_max: int = 4,
    mass_u: float = 15.1052848671,
    energy_kelvin: float = 3700, 
    rot_const = 9.243165268327e-7,
    wave_r0: float = 30,
    wave_r_sigma: float = 0.6,
    animation: AnimationConfig = AnimationConfig.No,
    frames: int = 60,
    wave_prefix: str = "coriolis",
    potential_path = "potentials/",
    transform_xpi: Optional[Callable[[Floating, Floating], Floating]] = None,
    transform_bsigma: Optional[Callable[[Floating, Floating], Floating]] = None,
    transform_api: Optional[Callable[[Floating, Floating], Floating]] = None,
    transform_potential: Optional[Callable[[Floating, Floating], Floating]] = None,
) -> split.Propagation:
    """
    Prepares the split operator propagation of the Ne OCS problem.

    :j_init: initial angular momentum of the OCS molecule
    :omega_init: initial body fixed projection of the OCS angular momentum equal to projection of the total angular momentum
    :j_tot: total angular momentum of the system
    :time_step: time step in the Hartree units of the propagation step
    :steps_no: number of steps in the propagation
    :r_start: starting value of the radial grid
    :r_end: ending value of the radial grid
    :r_no: number of points of the radial grid
    :polar_no: number of points of the angular grid
    :omega_max: maximum value of omega in the grid
    :mass_u: reduced mass of the Ne OCS in u units
    :energy_kelvin: energy of the collision in Kelvin units
    :rot_const: rotational constant of the OCS molecule in Hartree units
    :wave_r0: initial radial position of the wave funciton
    :wave_r_sigma: initial radial width of the wave funciton
    :animation: whether to save animations of the wave function
    :frames: number of frames of the animation
    :wave_prefix: prefix of the wave animation filename to be saved
    :wave_legendre_prefix: prefix of the wave animation in the legendre basis filename to be saved
    :potential_path: path to the potential data
    :transform_xpi: custom transformation to the XPi gamma potential
    :transform_bsigma: custom transformation to the BSigma gamma potential
    :transform_api: custom transformation to the APi gamma potential
    :transform_potential: custom transformation to the interatomic potential
    """

    match omega_init:
        case int():
            assert j_tot >= omega_init
            assert j_init >= omega_init
        case SpinOne():
            assert j_tot >= 1
            assert j_init == 1
        case _:
            raise Exception

    omega_max = min(omega_max, j_tot)

    ############################ grids, wave function creation ################################
    
    time_grid = split.TimeGrid(time_step, steps_no)
    r_grid = split.Grid.linear_continuos("r", r_start, r_end, r_no, 0)
    
    polar_points, weights = roots_legendre(polar_no)
    polar_points = np.flip(np.arccos(polar_points))
    weights = np.flip(weights)
    
    polar_grid = split.Grid.custom("theta", polar_points, weights, 1)

    omega_no = 2 * omega_max + 1
    omega_shift = omega_max
    omega_dim_nr = 2
    omega_grid = split.Grid.linear_countable("omega", -omega_max, omega_max, omega_no, omega_dim_nr)

    r_points = np.array(r_grid.points())
    momentum = np.sqrt(2 * mass_u * U * energy_kelvin * KELVIN)

    wave_r_init = np.array([split.gaussian_distribution(r_points[i], wave_r0, wave_r_sigma, momentum) for i in range(r_no)])

    wave_full_init = np.zeros((r_no, polar_no, omega_no), dtype=complex)
    match omega_init:
        case int():
            wave_polar_init = lpmv(omega_init, j_init, np.cos(polar_points))
            wave_init = np.outer(wave_r_init, wave_polar_init)
            wave_full_init[:, :, omega_shift+omega_init] = wave_init
        case SpinOne():
            population_zero = 1 - omega_init.population_1 - omega_init.population_minus_1
            wave_polar_init = np.sqrt(3 / 2 * population_zero) * lpmv(0, 1, np.cos(polar_points))
            wave_init = np.outer(wave_r_init, wave_polar_init)
            wave_full_init[:, :, omega_shift] = wave_init

            wave_polar_init = np.sqrt(3 / 4 * omega_init.population_1) * lpmv(1, 1, np.cos(polar_points))
            wave_init = np.outer(wave_r_init, wave_polar_init)
            wave_full_init[:, :, omega_shift+1] = wave_init * np.exp(1j * omega_init.phase_proj_1)

            wave_polar_init = np.sqrt(3 * omega_init.population_minus_1) * lpmv(-1, 1, np.cos(polar_points))
            wave_init = np.outer(wave_r_init, wave_polar_init)
            wave_full_init[:, :, omega_shift-1] = wave_init * np.exp(1j * omega_init.phase_proj_minus_1)

    wave_function = split.WaveFunction(wave_full_init.flatten(), [r_grid, polar_grid, omega_grid])

    ############################ operation creation ################################

    r_mesh, polar_mesh = np.meshgrid(r_points, polar_points, indexing="ij")

    potential_ab = np.zeros_like(wave_full_init)
    potential_xpi = np.zeros_like(wave_full_init)

    potential = load_potential(potential_path, "potential.dat", r_grid, polar_grid, 5, 5, False)
    if transform_potential is not None:
        potential = np.multiply(potential, transform_potential(r_mesh, polar_mesh))

    bsigma_gamma = load_potential(potential_path, "BSigma_gamma.dat", r_grid, polar_grid, 5, 5, True)
    if transform_bsigma is not None:
        bsigma_gamma = np.multiply(bsigma_gamma, transform_bsigma(r_mesh, polar_mesh))

    api_gamma = load_potential(potential_path, "APi_gamma.dat", r_grid, polar_grid, 5, 5, True)
    if transform_api is not None:
        api_gamma = np.multiply(api_gamma, transform_api(r_mesh, polar_mesh))

    xpi_gamma = load_potential(potential_path, "XPi_gamma.dat", r_grid, polar_grid, 5, 3, True)
    if transform_xpi is not None:
        xpi_gamma = np.multiply(xpi_gamma, transform_xpi(r_mesh, polar_mesh))

    for omega in range(-omega_max, omega_max + 1):
        centrifugal_potential = centrifugal(r_points, j_tot, omega, mass_u)
        centrifugal_potential = np.broadcast_to(np.expand_dims(centrifugal_potential, 1), (r_no, polar_no))
        
        potential_ab[:, :, omega_shift+omega] = potential + centrifugal_potential + complex(0, -0.5) * (bsigma_gamma + api_gamma)

        potential_xpi[:, :, omega_shift+omega] = complex(0, -0.5) * xpi_gamma

    potential_with_bsigma_prop = split.complex_n_dim_into_propagator(potential_ab.shape, potential_ab.flatten(), time_grid)

    loss_checker = split.LossChecker.new_with_saver("bsigma", frames, f"{wave_prefix}_bsigma", time_grid) if animation else split.LossChecker("bsigma")
    potential_with_bsigma_prop.set_loss_checked(loss_checker)

    xpi_gamma_prop = split.complex_n_dim_into_propagator(potential_xpi.shape, potential_xpi.flatten(), time_grid)
    
    loss_checker = split.LossChecker.new_with_saver("xpi", frames, f"{wave_prefix}_xpi", time_grid) if animation else split.LossChecker("xpi")
    xpi_gamma_prop.set_loss_checked(loss_checker)

    leak_control = split.LeakControl(split.LossChecker("leak control"))
    dumping_border = split.BorderDumping(5., 1., r_grid)

    angular_transformation = split.associated_legendre_transformations(polar_grid, omega_grid)

    shape, angular_kinetic_op = split.rotational_hamiltonian(r_grid, polar_grid, mass_u, rot_const)
    angular_op = np.broadcast_to(np.expand_dims(np.array(angular_kinetic_op).reshape(shape), 2), (r_no, polar_no, omega_no))

    angular_prop = split.n_dim_into_propagator(angular_op.shape, angular_op.flatten(), time_grid)

    ### coriolis
    j_grid = angular_transformation.transformed_grid()
    coriolis = split.NonDiagPropagator.get_coriolis(r_grid, j_grid, omega_grid, mass_u, j_tot, time_grid, step="half")
    ###

    fft_transformation = split.FFTTransformation(r_grid, "r momentum")

    kinetic_op = split.kinetic_hamiltonian(r_grid, mass_u, energy_kelvin)
    kinetic_prop = split.one_dim_into_propagator(kinetic_op, r_grid, time_grid, step = "full")

    ################################ populating operation stack ####################################

    operation_stack = split.OperationStack()
    potential_with_bsigma_prop.add_operation(operation_stack)
    xpi_gamma_prop.add_operation(operation_stack)

    if AnimationConfig.Distance in animation:
        wave_saver = split.StateSaver(data_path, f"{wave_prefix}_distance_animation", time_grid, r_grid, frames)
        wave_saver.add_operation(operation_stack)

    if AnimationConfig.Polar in animation:
        polar_saver = split.StateSaver(data_path, f"{wave_prefix}_polar_animation", time_grid, polar_grid, frames)
        polar_saver.add_operation(operation_stack)

    if AnimationConfig.AngProjection in animation:
        omega_saver = split.StateSaver(data_path, f"{wave_prefix}_omega_animation", time_grid, omega_grid, frames)
        omega_saver.add_operation(operation_stack)

    dumping_border.add_operation(operation_stack)
    leak_control.add_operation(operation_stack)

    j_grid = angular_transformation.transformed_grid()
    angular_transformation.add_operation(operation_stack, True)

    if AnimationConfig.Angular in animation:
        angular_saver = split.StateSaver(data_path, f"{wave_prefix}_angular_animation", time_grid, j_grid, frames)
        angular_saver.add_operation(operation_stack)

    angular_prop.add_operation(operation_stack)
    coriolis.add_operation(operation_stack)

    fft_transformation.add_operation(operation_stack, True)
    kinetic_prop.add_operation(operation_stack)

    ################################ propagation creation ####################################

    propagation = split.Propagation()
    propagation.set_wave_function(wave_function)
    propagation.set_time_grid(time_grid)
    propagation.set_operation_stack(operation_stack)

    return propagation

def default_value(name: str):
    import inspect

    return inspect.signature(prepare).parameters.get(name).default

class CumulativeLosses:
    def __init__(self, j_init: int, energy_kelvin: float = default_value("energy_kelvin")) -> None:
        self.bsigma_losses = []
        self.xpi_losses = []

        self.j_totals = [int(j_init + np.ceil(i * 5.5 * np.sqrt(energy_kelvin / 2000))) for i in range(50)]
    
    def extract_loss(self, propagation: split.Propagation) -> None:
        losses = propagation.get_losses()

        self.bsigma_losses.append(losses[0])
        self.xpi_losses.append(losses[1])

    def save_losses(self, filename: str) -> None:
        combined = np.vstack((self.j_totals, self.bsigma_losses, self.xpi_losses)).transpose()
        np.savetxt(f"{data_path}{filename}.dat", combined, delimiter=" ", header="j_total\tbsigma_loss\txpi_loss")

# Animations

In [3]:
print("j init: 0, omega init: 0, j_tot: 150")
propagation = prepare(0, 0, 150, animation=AnimationConfig.All, wave_prefix="coriolis_0_0", transform_api= lambda a,b: 0, transform_bsigma=lambda a,b: 0, transform_xpi=lambda a,b: 0)

propagation.propagate()
propagation.get_losses()
propagation.save_savers()

print("j init: 1, omega init: 0, j_tot: 150")
propagation = prepare(1, 0, 150, animation=AnimationConfig.All, wave_prefix="coriolis_1_0", transform_api= lambda a,b: 0, transform_bsigma=lambda a,b: 0, transform_xpi=lambda a,b: 0)

propagation.propagate()
propagation.get_losses()
propagation.save_savers()

print("j init: 1, omega init: 1, j_tot: 150")
propagation = prepare(1, 1, 150, animation=AnimationConfig.All, wave_prefix="coriolis_1_1", transform_api= lambda a,b: 0, transform_bsigma=lambda a,b: 0, transform_xpi=lambda a,b: 0)

propagation.propagate()
propagation.get_losses()
propagation.save_savers()

j init: 0, omega init: 0, j_tot: 150
populating coriolis


100%|██████████| 1024/1024 [00:19<00:00, 52.21it/s]


j init: 1, omega init: 0, j_tot: 150
populating coriolis


100%|██████████| 1024/1024 [00:18<00:00, 53.97it/s]


j init: 1, omega init: 1, j_tot: 150
populating coriolis


100%|██████████| 1024/1024 [00:18<00:00, 54.92it/s]


In [3]:
propagation = prepare(0, 0, 150, animation=AnimationConfig.Polar | AnimationConfig.AngProjection, wave_prefix="coriolis_0", transform_api= lambda a,b: 0, transform_bsigma=lambda a,b: 0, transform_xpi=lambda a,b: 0)

propagation.propagate()
propagation.get_losses()
propagation.save_savers()

populating coriolis


 38%|███▊      | 389/1024 [00:15<00:21, 30.08it/s]

In [7]:
propagation = prepare(1, 0, 200, omega_max=1, animation=AnimationConfig.All, wave_prefix="free_coriolis", transform_potential=lambda a,b: 0, transform_api= lambda a,b: 0, transform_bsigma=lambda a,b: 0, transform_xpi=lambda a,b: 0)

propagation.propagate()
propagation.get_losses()
propagation.save_savers()

0.00025610000011511147


In [3]:
def omega_1_phase_different(phase_1, phase_minus_1, suffix, omega_max=4):
    omega_init = SpinOne(1/3, phase_1, 1/3, phase_minus_1)
    propagation = prepare(1, omega_init, 150, omega_max=omega_max, animation=AnimationConfig.Polar | AnimationConfig.AngProjection, wave_prefix=f"coriolis_1_phase_{suffix}", transform_api= lambda a,b: 0, transform_bsigma=lambda a,b: 0, transform_xpi=lambda a,b: 0)

    propagation.propagate()
    propagation.get_losses()
    propagation.save_savers()

def omega_1_phase_reaction(phase_1, phase_minus_1, suffix):
    omega_init = SpinOne(1/3, phase_1, 1/3, phase_minus_1)
    propagation = prepare(1, omega_init, 150, wave_prefix=f"coriolis_1_phase_{suffix}")

    propagation.propagate()
    print(propagation.get_losses())

In [4]:
omega_1_phase_different(0, 0, "0_0")
omega_1_phase_different(np.pi, 0, "pi_0")
omega_1_phase_different(0, np.pi, "0_pi")
omega_1_phase_different(np.pi, np.pi, "pi_pi")
omega_1_phase_different(np.pi/2, 0, "hpi_0")
omega_1_phase_different(0, np.pi/2, "0_hpi")
omega_1_phase_different(np.pi/2, np.pi/2, "hpi_hpi")
omega_1_phase_different(np.pi, np.pi/2, "pi_hpi")
omega_1_phase_different(np.pi/2, np.pi, "hpi_pi")

populating coriolis


100%|██████████| 1024/1024 [00:36<00:00, 28.27it/s]


populating coriolis


100%|██████████| 1024/1024 [00:41<00:00, 24.52it/s]


populating coriolis


...

100%|██████████| 1024/1024 [00:36<00:00, 28.07it/s]


populating coriolis


100%|██████████| 1024/1024 [00:37<00:00, 27.59it/s]


populating coriolis


100%|██████████| 1024/1024 [00:38<00:00, 26.27it/s]


In [4]:
for omega_max in [1, 4, 10, 20]:
    omega_1_phase_different(0, 0, f"0_0_omega_max_{omega_max}", omega_max=omega_max)

populating coriolis


100%|██████████| 1024/1024 [00:22<00:00, 45.40it/s]


populating coriolis


100%|██████████| 1024/1024 [00:30<00:00, 33.64it/s]


populating coriolis


100%|██████████| 1024/1024 [01:01<00:00, 16.63it/s]


populating coriolis


100%|██████████| 1024/1024 [04:35<00:00,  3.72it/s]


# Single losses

In [ ]:
print("j init: 0, omega init: 0, j_tot: 0")
propagation = prepare(0, 0, 0)
propagation.propagate()
print(propagation.get_losses())

print("j init: 1, omega init: 0, j_tot: 1")
propagation = prepare(1, 0, 0)
propagation.propagate()
print(propagation.get_losses())

print("j init: 1, omega init: 1, j_tot: 1")
propagation = prepare(1, 1, 1)
propagation.propagate()
print(propagation.get_losses())

j init: 0, omega init: 0, j_tot: 150
[nan, nan]
j init: 1, omega init: 0, j_tot: 150


In [ ]:
print("j init: 0, omega init: 0, j_tot: 150")
propagation = prepare(0, 0, 150)
propagation.propagate()
print(propagation.get_losses())

omega_1_phase_reaction(0, 0, "0_0")
omega_1_phase_reaction(np.pi, 0, "pi_0")
omega_1_phase_reaction(0, np.pi, "0_pi")
omega_1_phase_reaction(np.pi, np.pi, "pi_pi")
omega_1_phase_reaction(np.pi/2, 0, "hpi_0")
omega_1_phase_reaction(0, np.pi/2, "0_hpi")
omega_1_phase_reaction(np.pi/2, np.pi/2, "hpi_hpi")
omega_1_phase_reaction(np.pi, np.pi/2, "pi_hpi")
omega_1_phase_reaction(np.pi/2, np.pi, "hpi_pi")

# Single cross sections

In [7]:
j_init = 0
omega_init = 0
energy_kelvin = 3700
omega_max = 1

losses = CumulativeLosses(j_init, energy_kelvin)

for j_tot in tqdm(losses.j_totals):
    propagation = prepare(j_init, omega_init, j_tot, omega_max=omega_max, energy_kelvin=energy_kelvin)
    propagation.propagate()
    losses.extract_loss(propagation)
losses.save_losses(f"coriolis_losses_omega_max_{omega_max}_{j_init}_{omega_init}")

100%|██████████| 50/50 [36:19<00:00, 43.60s/it]


In [8]:
j_init = 1
omega_init = 0
energy_kelvin = 3700
omega_max = 1
losses = CumulativeLosses(j_init, energy_kelvin)

for j_tot in tqdm(losses.j_totals):
    propagation = prepare(j_init, omega_init, j_tot, omega_max=omega_max, energy_kelvin=energy_kelvin)
    propagation.propagate()
    losses.extract_loss(propagation)
losses.save_losses(f"coriolis_losses_omega_max_{omega_max}_{j_init}_{omega_init}")

100%|██████████| 50/50 [36:26<00:00, 43.72s/it]


In [5]:
j_init = 1
omega_init = 1
energy_kelvin = 3700
omega_max = 1
losses = CumulativeLosses(j_init, energy_kelvin)

for j_tot in tqdm(losses.j_totals):
    propagation = prepare(j_init, omega_init, j_tot, omega_max=omega_max, energy_kelvin=energy_kelvin)
    propagation.propagate()
    losses.extract_loss(propagation)
losses.save_losses(f"coriolis_losses_omega_max_{omega_max}_{j_init}_{omega_init}")

100%|██████████| 50/50 [35:53<00:00, 43.07s/it]


In [6]:
j_init = 1
omega_init = SpinOne(1/3, 0, 1/3, 0)
energy_kelvin = 3700
omega_max = 2
losses = CumulativeLosses(j_init, energy_kelvin)

for j_tot in tqdm(losses.j_totals):
    propagation = prepare(j_init, omega_init, j_tot, omega_max=omega_max, energy_kelvin=energy_kelvin)
    propagation.propagate()
    losses.extract_loss(propagation)
losses.save_losses(f"coriolis_losses_omega_max_{omega_max}_{j_init}")

100%|██████████| 50/50 [35:41<00:00, 42.82s/it]


# Cross sections - energy dependence 

In [7]:
energies = [2000, 2500, 3000, 3500, 4000, 4500, 5000]

In [9]:
for energy_kelvin in energies:
    print("energy", energy_kelvin)
    j_init = 0
    omega_init = 0
    time_step = default_value("time_step")*np.sqrt(energy_kelvin/default_value("energy_kelvin"))
    losses = CumulativeLosses(j_init, energy_kelvin)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, energy_kelvin=energy_kelvin, time_step=time_step)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"losses_{energy_kelvin}_{j_init}_{omega_init}")

energy 100


  4%|▍         | 2/50 [01:05<26:07, 32.66s/it]

In [ ]:
for energy_kelvin in energies:
    print("energy", energy_kelvin)
    j_init = 1
    omega_init = 0
    time_step = default_value("time_step")*np.sqrt(energy_kelvin/default_value("energy_kelvin"))
    losses = CumulativeLosses(j_init, energy_kelvin)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, energy_kelvin=energy_kelvin, time_step=time_step)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"losses_{energy_kelvin}_{j_init}_{omega_init}")

energy 100


100%|██████████| 50/50 [04:20<00:00,  5.22s/it]


energy 500


100%|██████████| 50/50 [04:21<00:00,  5.22s/it]


energy 1000


...

100%|██████████| 50/50 [04:26<00:00,  5.33s/it]


energy 4500


100%|██████████| 50/50 [04:22<00:00,  5.24s/it]


energy 5000


100%|██████████| 50/50 [04:22<00:00,  5.24s/it]


In [ ]:
for energy_kelvin in energies:
    print("energy", energy_kelvin)
    j_init = 1
    omega_init = 1
    time_step = default_value("time_step")*np.sqrt(energy_kelvin/default_value("energy_kelvin"))
    losses = CumulativeLosses(j_init, energy_kelvin)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, energy_kelvin=energy_kelvin, time_step=time_step)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"losses_{energy_kelvin}_{j_init}_{omega_init}")

energy 100


100%|██████████| 50/50 [03:36<00:00,  4.33s/it]


energy 500


100%|██████████| 50/50 [03:35<00:00,  4.30s/it]


energy 1000


...

100%|██████████| 50/50 [03:31<00:00,  4.24s/it]


energy 4500


100%|██████████| 50/50 [03:30<00:00,  4.21s/it]


energy 5000


100%|██████████| 50/50 [03:34<00:00,  4.28s/it]


# Convergences 

## Time step

In [14]:
time_steps = [50, 100, 200, 400, 600, 1000, 2000, 4000, 6000]

In [15]:
for time_step in time_steps:
    print("time_step", time_step)
    j_init = 0
    omega_init = 0
    steps_no = max(int(default_value("steps_no") * default_value("time_step") / time_step), 100)
    losses = CumulativeLosses(j_init)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, time_step=time_step, steps_no=steps_no)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"time_step_{time_step}_{j_init}_{omega_init}")

time_step 50


100%|██████████| 50/50 [10:24<00:00, 12.49s/it]


time_step 100


100%|██████████| 50/50 [05:11<00:00,  6.23s/it]


time_step 200


...

100%|██████████| 50/50 [00:27<00:00,  1.80it/s]


time_step 4000


100%|██████████| 50/50 [00:28<00:00,  1.77it/s]


time_step 6000


100%|██████████| 50/50 [00:28<00:00,  1.77it/s]


In [16]:
for time_step in time_steps:
    print("time_step", time_step)
    j_init = 1
    omega_init = 0
    steps_no = max(int(default_value("steps_no") * default_value("time_step") / time_step), 100)
    losses = CumulativeLosses(j_init)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, time_step=time_step, steps_no=steps_no)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"time_step_{time_step}_{j_init}_{omega_init}")

time_step 50


100%|██████████| 50/50 [10:17<00:00, 12.35s/it]


time_step 100


100%|██████████| 50/50 [05:03<00:00,  6.08s/it]


time_step 200


...

100%|██████████| 50/50 [00:28<00:00,  1.76it/s]


time_step 4000


100%|██████████| 50/50 [00:27<00:00,  1.79it/s]


time_step 6000


100%|██████████| 50/50 [00:28<00:00,  1.78it/s]


In [17]:
for time_step in time_steps:
    print("time_step", time_step)
    j_init = 1
    omega_init = 1
    steps_no = max(int(default_value("steps_no") * default_value("time_step") / time_step), 100)
    losses = CumulativeLosses(j_init)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, time_step=time_step, steps_no=steps_no)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"time_step_{time_step}_{j_init}_{omega_init}")

time_step 50


100%|██████████| 50/50 [10:11<00:00, 12.24s/it]


time_step 100


100%|██████████| 50/50 [05:10<00:00,  6.22s/it]


time_step 200


...

100%|██████████| 50/50 [00:30<00:00,  1.66it/s]


time_step 4000


100%|██████████| 50/50 [00:29<00:00,  1.72it/s]


time_step 6000


100%|██████████| 50/50 [00:28<00:00,  1.73it/s]


## radial grid

In [ ]:
r_nos = [128, 256, 512, 1024, 2048, 4096]

In [ ]:
for r_no in r_nos:
    print("radial_no", r_no)
    j_init = 0
    omega_init = 0
    losses = CumulativeLosses(j_init)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, r_no=r_no)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"radial_scaling_{r_no}_{j_init}_{omega_init}")

In [ ]:
for r_no in r_nos:
    print("radial_no", r_no)
    j_init = 1
    omega_init = 0
    losses = CumulativeLosses(j_init)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, r_no=r_no)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"radial_scaling_{r_no}_{j_init}_{omega_init}")

In [ ]:
for r_no in r_nos:
    print("radial_no", r_no)
    j_init = 1
    omega_init = 1
    losses = CumulativeLosses(j_init)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, r_no=r_no)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"radial_scaling_{r_no}_{j_init}_{omega_init}")

## polar grid

In [ ]:
polar_nos = [20, 40, 80, 160, 320, 640]

In [ ]:
for polar_no in polar_nos:
    print("polar_no", polar_no)
    j_init = 0
    omega_init = 0
    losses = CumulativeLosses(j_init)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, polar_no=polar_no)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"polar_scaling_{polar_no}_{j_init}_{omega_init}")

In [ ]:
for polar_no in polar_nos:
    print("polar_no", polar_no)
    j_init = 1
    omega_init = 0
    losses = CumulativeLosses(j_init)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, polar_no=polar_no)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"polar_scaling_{polar_no}_{j_init}_{omega_init}")

In [ ]:
for polar_no in polar_nos:
    print("polar_no", polar_no)
    j_init = 1
    omega_init = 1
    losses = CumulativeLosses(j_init)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, polar_no=polar_no)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"polar_scaling_{polar_no}_{j_init}_{omega_init}")

## wave width

In [ ]:
widths = [0.1, 0.3, 0.6, 1, 2, 4]

In [ ]:
for width in widths:
    print("wave width", width)
    j_init = 0
    omega_init = 0
    losses = CumulativeLosses(j_init)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, wave_r_sigma=width)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"width_scaling_{width}_{j_init}_{omega_init}")

In [ ]:
for width in widths:
    print("wave width", width)
    j_init = 1
    omega_init = 0
    losses = CumulativeLosses(j_init)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, wave_r_sigma=width)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"width_scaling_{width}_{j_init}_{omega_init}")

In [ ]:
for width in widths:
    print("wave width", width)
    j_init = 1
    omega_init = 1
    losses = CumulativeLosses(j_init)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, wave_r_sigma=width)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"width_scaling_{width}_{j_init}_{omega_init}")